# 2. After tximport in R

# 3. Merge annotations for each species, connecting Trinity contigs/genes to annotation

In [1]:
import os
import pandas as pd
# requires dammit env
# source activate dammit
from dammit.fileio.gff3 import GFF3Parser

In [4]:
counts_matrices = "/Users/johnsolk/Documents/UCDavis/Whitehead/counts_matrices/"
counts_files = os.listdir(counts_matrices)
gene_out_dir = "/Users/johnsolk/Documents/UCDavis/Whitehead/contig_gene_name_23June2018/"

In [12]:
for counts_file in counts_files:
    if counts_file != ".DS_Store":
        species = counts_file.split("_")[0]+"_"+counts_file.split("_")[1]
        print("========")
        print(species)
        print("========")
        gene_out = gene_out_dir + species + "_gene_counts_annotations.csv"
        table = pd.read_csv("/Users/johnsolk/Documents/UCDavis/Whitehead/counts_matrices/"+counts_file)
        print('Number of Trinity "genes" (this is how we summarized expression):')
        print(table.shape)
        table = table.rename(columns={'Unnamed: 0': 'Gene'})
        table_filt = table[table.sum(axis=1) > 1] 
        #table_filt = table[(table > 0).sum(axis=1) >= 5]
        #https://stackoverflow.com/questions/34712050/filter-by-row-sum-and-value
        #table_filt = table [table.mask(table!=0).count(axis=1).div(float(len(table.columns))) < 0.05]
        print('Contigs with rowSum expression > 1')
        print(table_filt.shape)
        name = "/Users/johnsolk/Documents/UCDavis/Whitehead/gff_annotations/"+species+".trinity_out.Trinity.fasta.dammit.gff3"
        conversion_contig = "/Users/johnsolk/Documents/UCDavis/Whitehead/contig_gene_23June2018/"+species+"_contig_gene.csv"
        conversion_dammit = "/Users/johnsolk/Documents/UCDavis/Whitehead/dammit_conversions/"+species+".trinity_out.Trinity.fasta.dammit.namemap.csv"
        annotations = GFF3Parser(filename=name).read()
        annotations = annotations.dropna(subset=['Name'])
        annotations["length"] = annotations["end"].subtract(annotations["start"], fill_value=0)
        pickonename = annotations.sort_values(by=['seqid', 'score'], ascending=True).query('score < 1e-05').drop_duplicates(subset='seqid')[['seqid', 'Name','Note','database','Dbxref','start','end','length']]
        pickonename = pickonename.dropna(axis=0,how="all")
        print('Number of contigs with annotations (one annotation/contig, sorted by E-value < 1e-05 and picked the lowest):')
        print(pickonename.shape)
        conversions_dammit = pd.read_csv(conversion_dammit)
        conversions_contig = pd.read_csv(conversion_contig)
        conversions_dammit['Name'], conversions_dammit['info'] = conversions_dammit['original'].str.split(' ', 1).str
        conversions_dammit = conversions_dammit[['Name','renamed']]
        conversions_dammit.columns = ['Name','seqid']
        coversions_contig = conversions_contig[['Name','Gene']]
        merged_table = pd.merge(table_filt,conversions_contig,on="Gene")
        merged_table = pd.merge(merged_table,conversions_dammit,on="Name")
        merged_table = pd.merge(merged_table,pickonename,on="seqid")
        print('Unique gene names in contigs with expression:')
        print(len(merged_table.Name_y.unique()))
        fhet = annotations[annotations['Name'].str.startswith("gi")]
        fhet_filtered = fhet.query('score < 1e-05').drop_duplicates(subset='seqid')[['seqid', 'Name','start','end','length']]
        print("Unique Fhet gene names (one name per contig):")
        print(len(fhet_filtered.Name.unique()))
        fhet_merged_table = pd.merge(merged_table,fhet_filtered,on='seqid',how='outer')
        fhet_merged_table = fhet_merged_table.rename(columns = {'Gene':'TrinityGene','Name_x':'TrinityContig','seqid':'dammitSeqID','Name_y':'GeneName','start_x':'annotationStart','end_x':'annotationEnd','length_x':'annotationLength','Name':'FhetNCBIName','start_y':'FhetNCBIStart','end_y':'FhetNCBIEnd','length_y':'FhetNCBILength'})                                           
        fhet_merged_table = fhet_merged_table.drop('Unnamed: 0', 1)
        fhet_merged_table['split1'], fhet_merged_table['split2'],fhet_merged_table['split3'],fhet_merged_table['NCBIproteinID'],fhet_merged_table['NCBIproteinName'] = fhet_merged_table['FhetNCBIName'].str.split('|', 5).str
        fhet_merged_table = fhet_merged_table.drop('split1',1)
        fhet_merged_table = fhet_merged_table.drop('split2',1)
        fhet_merged_table = fhet_merged_table.drop('split3',1)
        print("Unique Fhet annotations, contigs with expression")
        print(len(fhet_merged_table.FhetNCBIName.unique()))
        print('Unique NCBI protein ID')
        print(len(fhet_merged_table.NCBIproteinID.unique()))
        print('Unique annotated gene names')
        print(len(fhet_merged_table.GeneName.unique()))
        print('Unique Trinity "genes"')
        print(len(fhet_merged_table.TrinityGene.unique()))
        fhet_merged_table.to_csv(gene_out)
    

F_heteroclitusMDPP
Number of Trinity "genes" (this is how we summarized expression):
(496133, 10)
Contigs with rowSum expression > 1
(450559, 10)


/Users/johnsolk/anaconda3/lib/python3.6/site-packages/dammit/fileio/gff3.py:73: ParserWarning: Both a converter and dtype were specified for column attributes - only the converter will be used
  dtype=dict(self.columns)):


Number of contigs with annotations (one annotation/contig, sorted by E-value < 1e-05 and picked the lowest):
(186798, 8)
Unique gene names in contigs with expression:
53546
Unique Fhet gene names (one name per contig):
22994
Unique Fhet annotations, contigs with expression
22995
Unique NCBI protein ID
22995
Unique annotated gene names
53547
Unique Trinity "genes"
79551
F_parvapinis
Number of Trinity "genes" (this is how we summarized expression):
(279009, 9)
Contigs with rowSum expression > 1
(256458, 9)
Number of contigs with annotations (one annotation/contig, sorted by E-value < 1e-05 and picked the lowest):
(126200, 8)
Unique gene names in contigs with expression:
45656
Unique Fhet gene names (one name per contig):
20647
Unique Fhet annotations, contigs with expression
20648
Unique NCBI protein ID
20648
Unique annotated gene names
45657
Unique Trinity "genes"
60406
L_parva
Number of Trinity "genes" (this is how we summarized expression):
(275950, 10)
Contigs with rowSum expression 

F_similis
Number of Trinity "genes" (this is how we summarized expression):
(392860, 10)
Contigs with rowSum expression > 1
(356644, 10)
Number of contigs with annotations (one annotation/contig, sorted by E-value < 1e-05 and picked the lowest):
(154675, 8)
Unique gene names in contigs with expression:
47743
Unique Fhet gene names (one name per contig):
22338
Unique Fhet annotations, contigs with expression
22339
Unique NCBI protein ID
22339
Unique annotated gene names
47744
Unique Trinity "genes"
67283
